<a href="https://colab.research.google.com/github/FranzDS01/InteligenciaArtifical/blob/main/Segundo%20Parcial/MulticlaseSegundoParcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### One vs all
Nombre:Soria Colque Franz Ronald

Carrera:Ing de Sistemas

Repositorio:

### Problema

* **dataset:** https://www.kaggle.com/datasets/saurabhshahane/adl-classification

* **Descriccion:**Prediccion de clasificacion de la calidad de aire

* **Objetivo:**:Calidad de aire

* **Detalle de Solucion:**:Primero se hace tratado de datos, luego como el dataset tiene pocos datos se genera nuevo  datos sinteticos

  **Funciones Implentatadas:**

  **tratadoDAtos()**:Transforma las columnas type[objet] a numericas,Rellena datos nulos y elimna columnas inesesarias

  **reorganizarDataset()**:coloca la columna objetivo 'y' al final del dataset
  
  **Segmentacion()**:Separa el dataset para datos de entrenamiento y prueba

  **listaDataframes()** crear una lista de Dataframes a partir del dataset original ,cada dataframe creado esta clasificado segun la columnas objetico 'y' si el dataset original tiene 3 clasificaciones entoces la lista tedra 3 Dataframes Dataframes1:clases 0 ,Dataframes2:clases 1 ,Dataframes3:clases 2

  **DatosSinteticos()**:Crear datos sinteticos por columnas , perimero se crear aleatoriamente la columnas las 'y'
  de (0,1,2)o (0,2,3,4) segun la cantidad de clasificaciones que tenga, para crear el resto de las columnas se usa las ListaDataframes , ejemplo:sin el primero valor de 'y' generado es clase 0 , para crear las columnas restantes'X' se usa El primer dataframe de la lista que pertence a esa clase 0
  
  **col_categoricas()**:retorna una lista de las posiciones de columnas categoricas
  

* **Mejores valores para prediccion del modelo:**

* **Lambda:**0.01

* **maxiter:**500

* **Mejores valores de theta encontrados:**
[[-3.34415954e-01  3.47078029e-03  2.26277666e-03 -2.74086324e-03
  -1.07234056e-02  6.81342066e-03  3.08791425e-03]

 [-1.37053756e+00  1.93326416e-03  4.83176447e-03  7.03960841e-03
   7.04839530e-03  2.01391642e-03 -1.42808362e-02]

 [ 2.69129105e+01 -3.25427758e-02 -1.93810835e-02 -1.28407377e-02
  -8.54531468e-03 -4.73765521e-03  1.38077393e-02]
  
 [-5.83552333e+00 -2.59255838e-03 -4.06739600e-03  3.97406618e-03
   4.64467091e-03 -2.84639988e-03  1.99520957e-04]]


In [ ]:
import os
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
%matplotlib inline

### Tratado de los datos 

In [ ]:
def tratadoDatos(dataset,modificarColumnas,eliminarColumnas):
    data =pd.read_csv(dataset,encoding='latin-1')
    columna = data.columns
    for index in modificarColumnas:
        name = columna[index]
        value = list(data[name].unique())
        data[name] = data[name].map(dict(zip(value,[i for i in range(len(value))])))
    #print(data.head())
    data = data.drop([columna[index]for index in eliminarColumnas], axis=1)
    valor_media=data.select_dtypes(include=['number']).mean()
    data = data.fillna(valor_media.round(2))
    return data

In [ ]:
def DatosSinteticos(data,listaData,n_filas,clases):
    nuevos_datos=np.empty([n_filas,len(data.columns)])
    ColmunaY = np.random.randint(0,clases, n_filas)
    #print(ColmunaY)
    for i in range(len(data.columns)-1):
        for j in range(0,n_filas):
            pos=ColmunaY[j]
            columna_i =listaData[pos].iloc[:,i]
            muestras_i=np.random.choice(columna_i,1,replace=True)

            pro_cambio = random.random()
            decision = random.randint(0, 1)
            if pro_cambio < 0.2:
               muestras_i = muestras_i + pro_cambio if decision == 0 else muestras_i - pro_cambio

            if(i == 0):
                muestras_i=muestras_i.round(5)
            else:
                muestras_i=muestras_i.round(2)
            #np.random.seed()
            nuevos_datos[j:,i]=muestras_i
            nuevos_datos[j:,len(data.columns)-1]=ColmunaY[j]
    columnas =list(data.columns)
    print(nuevos_datos)
    nuevos_datos_data=pd.DataFrame(nuevos_datos,columns=columnas)
    dataExtend =pd.concat([data,nuevos_datos_data],axis=0)
    #print(dataExtend.tail())
    return dataExtend

In [ ]:
def listaDataframes(data,clases,columna_y):
    ListaData=[]
    for i in range(0,clases):
        ListaData.append(data[data[columna_y] == i])
    return ListaData

In [ ]:
def col_categoricas(ruta):
    data =pd.read_csv(ruta)
    columnas_numericas = data.select_dtypes(include=['object'])
    col_numericas = np.array([data.columns.get_loc(col) for col in columnas_numericas.columns])
    return col_numericas

In [ ]:
def num_clasificaciones(data,columnay):
    value = list(data[columnay].unique())
    print("clases :",value)
    print('Cantidad:',len(value))
    data.info()

In [ ]:
def reorganizarDataset(data, posicion_columna):
    # Obtener el nombre de la columna a mover
    nombre_columna = data.columns[posicion_columna]

    # Crear una lista con las columnas en el nuevo orden
    columnas_nuevas = list(data.columns[:posicion_columna]) + list(data.columns[posicion_columna+1:]) + [nombre_columna]
    
    # Crear un nuevo DataFrame con las columnas reorganizadas
    data_reorganizado = data[columnas_nuevas]
    
    return data_reorganizado

### Algortimo one vs all

In [ ]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)

# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

In [ ]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    z=np.clip(z,-500,500)#para evitar que z se desvorde 
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
 
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size
    epsilon = 1e-15  #evitar diviciones entre 0
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h+epsilon)) - (1 - y).dot(np.log(1 - h+epsilon))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [ ]:
def oneVsAll(X, y, num_labels, lambda_,maxiter):
    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': maxiter}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == c), lambda_), 
                                jac=True, 
                                method='CG',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [ ]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]
    p = np.zeros(m)
    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)
    return p

### Preparar Dataset 

In [ ]:
data=pd.read_csv('./Dataset/calidadAire.csv')
#num_clasificaciones(data,'')

In [ ]:
print(data.columns)
data

Index(['A', 'B', 'C', 'D', 'E', 'F', 'calidad;;;;'], dtype='object')


,A,B,C,D,E,F,calidad;;;;
0,670,696,1252,1720,1321,2431,4;;;;
1,641,674,1156,1652,1410,2433,4;;;;
2,642,646,1159,1643,1455,2361,4;;;;
3,640,590,1105,1608,1459,2427,4;;;;
4,616,627,1192,1637,1466,2447,4;;;;
...,...,...,...,...,...,...,...
1840,862,826,1564,1768,1540,2037,4;;;;
1841,917,821,1571,1779,1543,2008,4;;;;
1842,925,832,1582,1776,1545,1989,4;;;;
1843,928,840,1587,1787,1538,1986,4;;;;


In [ ]:
#data = data.fillna(data.mean)
#data['salario'] = data['salario'].fillna(100) #esto es para rellenar valores nulos en una columna especifica
rutadatset='./Dataset/calidadAire.csv'
col_categoricas(rutadatset)
data=tratadoDatos(rutadatset,[6],[])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1845 entries, 0 to 1844
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   A            1845 non-null   int64
 1   B            1845 non-null   int64
 2   C            1845 non-null   int64
 3   D            1845 non-null   int64
 4   E            1845 non-null   int64
 5   F            1845 non-null   int64
 6   calidad;;;;  1845 non-null   int64
dtypes: int64(7)
memory usage: 101.0 KB


In [ ]:
data.head()

,A,B,C,D,E,F,calidad;;;;
0,670,696,1252,1720,1321,2431,0
1,641,674,1156,1652,1410,2433,0
2,642,646,1159,1643,1455,2361,0
3,640,590,1105,1608,1459,2427,0
4,616,627,1192,1637,1466,2447,0


In [ ]:

num_clasificaciones(data,'calidad;;;;')

clases : [0, 1, 2, 3]
Cantidad: 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1845 entries, 0 to 1844
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   A            1845 non-null   int64
 1   B            1845 non-null   int64
 2   C            1845 non-null   int64
 3   D            1845 non-null   int64
 4   E            1845 non-null   int64
 5   F            1845 non-null   int64
 6   calidad;;;;  1845 non-null   int64
dtypes: int64(7)
memory usage: 101.0 KB


##### reoriganizar y crear datos sinteticos si lo requiere

In [ ]:
#num_clasificaciones(data,'Segmentation')
#Reorganizar las columnas
#datar=reorganizarDataset(data,8)
#datar.head()
Listadata=listaDataframes(data,4,'calidad;;;;')
data=DatosSinteticos(data,Listadata,9000,4)
#generar un nuevo archivo cvs del nuevo dataset con datos sinteticos


[[3.3812096e+02 5.2700000e+02 8.8400000e+02 ... 1.1370000e+03
  2.3440000e+03 2.0000000e+00]
 [6.0000000e+02 7.3400000e+02 1.3531400e+03 ... 1.4140000e+03
  2.4520000e+03 0.0000000e+00]
 [5.6988430e+02 9.7300000e+02 1.4830000e+03 ... 1.5840000e+03
  2.0660000e+03 1.0000000e+00]
 ...
 [7.9800000e+02 8.6400000e+02 1.1550000e+03 ... 1.5180000e+03
  2.0300000e+03 0.0000000e+00]
 [3.8700000e+02 5.3300000e+02 1.0761600e+03 ... 9.3600000e+02
  2.2488200e+03 2.0000000e+00]
 [6.0000000e+02 7.0000000e+02 1.4280000e+03 ... 1.6270000e+03
  2.4940000e+03 0.0000000e+00]]


In [ ]:
data.to_csv('sinteticosCalidadAire.csv',index=False)

### segmentacion del DataFrame para entrenamiento y Prueba

In [ ]:
#separar columna objetivo del dataFrame para X  y asignar objetivo a Y
def segmantacion(data,columnay):
    X = data.drop(columnay, axis=1)
    y = data[columnay]
    X_entre, X_prueba, y_entre, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_entre.to_numpy(), y_entre.to_numpy(),X_prueba.to_numpy(),y_prueba.to_numpy()

In [ ]:
X_entre,y_entre,X_prueba,y_prueba=segmantacion(data,'calidad;;;;')
num_labels = 4
print('Entrenamiento:',X_entre.shape,y_entre.shape,'Prueba:',X_prueba.shape,y_prueba.shape)

Entrenamiento: (8676, 6) (8676,) Prueba: (2169, 6) (2169,)


### Ejecutar algoritmo One vs All

In [ ]:
lambda_ = 0.01 #mejor lambda es 0.009 maxiter=500
all_theta = oneVsAll(X_entre, y_entre, num_labels, lambda_,500)
print(all_theta.shape)
print(all_theta)

(4, 7)
[[-3.34415954e-01  3.47078029e-03  2.26277666e-03 -2.74086324e-03
  -1.07234056e-02  6.81342066e-03  3.08791425e-03]
 [-1.37053756e+00  1.93326416e-03  4.83176447e-03  7.03960841e-03
   7.04839530e-03  2.01391642e-03 -1.42808362e-02]
 [ 2.69129105e+01 -3.25427758e-02 -1.93810835e-02 -1.28407377e-02
  -8.54531468e-03 -4.73765521e-03  1.38077393e-02]
 [-5.83552333e+00 -2.59255838e-03 -4.06739600e-03  3.97406618e-03
   4.64467091e-03 -2.84639988e-03  1.99520957e-04]]


In [ ]:
print(X_prueba.shape)
pred = predictOneVsAll(all_theta, X_entre)
print('Precision del conjuto de entrenamiento: {:.4f}%'.format(np.mean(pred == y_entre) * 100))
fila=529 #527
Xp = X_prueba[fila:fila+1, :].copy()
print("se espera de clase :",y_prueba[fila])

Xp = np.concatenate([np.ones((1, 1)), Xp], axis=1)
p = np.argmax(sigmoid(Xp.dot(all_theta.T)), axis = 1)
print("La clase es: ", p)

(2169, 6)
Precision del conjuto de entrenamiento: 86.2840%
se espera de clase : 3.0
La clase es:  [3]
